In [1]:
import pyaudio
import threading
import time

import torch
import numpy as np

In [2]:
FORMAT = pyaudio.paInt16
CHANNELS = 1
SAMPLE_RATE = 16000
CHUNK = int(SAMPLE_RATE / 10)

audio = pyaudio.PyAudio()

ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5204:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5204:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1342:(snd_func_refer) error evaluating name
ALSA lib conf.c:5204:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5727:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM sysdefault
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5204:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5204:(_snd_config_evaluate) function snd_func_concat returned error: No

In [ ]:
model = torch.load("v5_1.pth", weights_only=False)

/home/akbar/vad_train/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'MyDatabase.SpeakerDiarization.MyProtocol' found in /home/akbar/vad_train/database.yml does not define the 'scope' of speaker labels (file, database, or global). Setting it to 'file'.


In [4]:
# Provided by Alexander Veysov
def int2float(sound):
    abs_max = np.abs(sound).max()
    sound = sound.astype('float32')
    if abs_max > 0:
        sound *= 1/32768
    sound = sound.squeeze()  # depends on the use case
    return sound

In [5]:
continue_recording = True

num_samples = 4096 * 2

def stop():
    input("Press Enter to stop the recording:")
    global continue_recording
    continue_recording = False

In [6]:
def start_recording_sliding_window():
    
    stream = audio.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=SAMPLE_RATE,
                    input=True,
                    frames_per_buffer=CHUNK)
    
    global continue_recording
    continue_recording = True
        
    stop_listener = threading.Thread(target=stop)
    stop_listener.start()

    audio_array = np.zeros(1)
    duration, step =int(5.0 * SAMPLE_RATE), int(1.5 * SAMPLE_RATE)
    current_point = 0

    pred_list = [] 
    
    start  = time.time()
    while continue_recording:
    
        audio_chunk = stream.read(num_samples)
        
        audio_int16 = np.frombuffer(audio_chunk, np.int16)
        audio_float32 = audio_int16.astype(np.float32) / 32767.0
        # print(f"Type: {type(audio_float32)}\nSize: {len(audio_float32)}")
        
        # audio_array at the end will contain wave array of whole audio
        audio_array = audio_float32 if audio_array.shape[0] == 1 else np.concatenate((audio_array, audio_float32), axis=0)

        if len(audio_array) - current_point < duration:
            continue
        
        audio_torch = torch.from_numpy(audio_array[current_point:current_point+duration])
        current_point += step

        # print("Started")
        with torch.no_grad():
            out = model(audio_torch.reshape(1, 1, -1)).squeeze(0, 2).numpy()
            preds = (out >= 0.8)
            pred_list.append(out)
            print(f"Prediction: {preds}")
    end = time.time()
    
    while current_point < len(audio_array):
        audio_torch = torch.from_numpy(audio_array[current_point:current_point+duration])
        current_point += step

        with torch.no_grad():
            out = model(audio_torch.reshape(1, 1, -1)).squeeze(0, 2).numpy()
            preds = (out >= 0.8)
            print(f"Prediction: {preds}")
        
    print(f"duration: {(end - start):.3f}")

In [7]:
start_recording_sliding_window()

Prediction: [False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False  True  True False False False
 False False False False False False False False False False  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  T

In [ ]:
def stop():
    input("Press Enter to stop the recording:")
    global continue_recording
    continue_recording = False

In [ ]:
def start_recording():
    
    stream = audio.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=SAMPLE_RATE,
                    input=True,
                    frames_per_buffer=CHUNK)
    
    global continue_recording
    continue_recording = True
        
    stop_listener = threading.Thread(target=stop)
    stop_listener.start()

    print(f"Start")
    while continue_recording:
    
        audio_chunk = stream.read(num_samples)
        
        audio_int16 = np.frombuffer(audio_chunk, np.int16)
        audio_float32 = audio_int16.astype(np.float32) / 32767.0        
        audio_torch = torch.from_numpy(audio_float32)
    
        with torch.no_grad():
            out = model(audio_torch.reshape(1, 1, -1)).squeeze(0, 2).numpy()
            preds = (out >= 0.8)
            print(f"Prediction: {preds}")
            
    print(f"End")

In [10]:
def stop():
    input("Press Enter to stop the recording:")
    global continue_recording
    continue_recording = False

In [ ]:
# def main():
    # parser = argparse.ArgumentParser(description='Create a voice activity detection dataset')
    # parser.add_argument('--num_samples', type=int, default=512, help='Set Number of samples. Min is 4096')
    # args = parser.parse_args()
      
stream = audio.open(format=FORMAT,
                channels=CHANNELS,
                rate=SAMPLE_RATE,
                input=True,
                frames_per_buffer=CHUNK)

global continue_recording
continue_recording = True
    
stop_listener = threading.Thread(target=stop)
stop_listener.start()

print(f"Start")

while continue_recording:

    audio_chunk = stream.read(4096)
    
    audio_int16 = np.frombuffer(audio_chunk, np.int16)
    audio_float32 = audio_int16.astype(np.float32) / 32767.0        
    audio_torch = torch.from_numpy(audio_float32)
    with torch.no_grad():
        out = model(audio_torch.reshape(1, 1, -1)).squeeze(0, 2).numpy()
        # yield out
        preds = (out >= 0.85)
        if preds.any(True):
            print(f"Prediction: {True}")
        else:
            print(f"Prediction: {False}")

Start
[0.40134737 0.30128017 0.26434973 0.24874628 0.24212398 0.23672298
 0.22469775 0.21374202 0.20991552 0.21309355 0.22539838 0.26033902]




AttributeError: 'numpy.ndarray' object has no attribute 'read'

In [19]:
main()

Start
tensor([ 6.1037e-05, -1.8006e-03, -3.9979e-03, -3.1434e-03, -2.8382e-03,
        -5.0050e-03, -7.1718e-03, -6.7751e-03, -4.1505e-03, -2.6551e-03,
        -1.7090e-03,  1.8311e-04,  1.5870e-03,  1.2207e-03, -2.7467e-04,
        -4.5778e-04,  3.9674e-04,  4.5778e-04, -4.5778e-04, -3.0519e-05,
         1.0071e-03,  1.0071e-03,  1.2207e-03,  9.7659e-04,  1.0681e-03,
         1.9532e-03,  1.5870e-03,  1.3733e-03,  2.1058e-03,  1.7090e-03,
         1.2207e-03,  1.8006e-03,  3.4486e-03,  4.6999e-03,  5.1576e-03,
         6.2868e-03,  6.1647e-03,  4.3947e-03,  3.7843e-03,  3.2960e-03,
         1.6175e-03,  1.0376e-03,  1.8311e-03,  2.9603e-03,  4.6693e-03,
         5.7375e-03,  4.8219e-03,  3.2960e-03,  2.0447e-03,  0.0000e+00,
        -1.9837e-03, -2.3194e-03, -1.7701e-03, -1.1902e-03, -1.0987e-03,
        -7.3244e-04, -6.7141e-04, -1.6785e-03, -2.8993e-03, -3.7538e-03,
        -4.1810e-03, -4.6693e-03, -4.6388e-03, -4.1505e-03, -3.4791e-03,
        -2.6551e-03, -2.1363e-03, -2.2889e-03

ValueError: Expected more than 1 spatial element when training, got input size torch.Size([1, 60, 1])